### Import

In [ ]:
import os
import numpy as np
from numpy import abs, max 
import scipy
from scipy import interpolate
from matplotlib import pyplot as plt
import sys
import pandas as pd
import random
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import glob

import keras
from keras.layers import Input,LSTM,Dense,Dropout,Activation,noise,normalization,TimeDistributed,Flatten,Masking,Embedding,Conv1D, MaxPooling1D, RepeatVector, Permute, Lambda, Multiply,Reshape
from keras.models import Sequential, Model, load_model
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import EarlyStopping
from IPython.display import SVG
import keras_utils
from keras.utils.np_utils import to_categorical
from sklearn.metrics import mean_absolute_error
from keras import backend as K

from tensorflow import keras

### Composite Data Creation Functions

In [ ]:
def synthesize_data(subject,action,num,times):
    
    base_path = '../data/one_time_action_opt/'+subject+'/'+action
    synthetic_data = np.zeros((3,0))
    begin = np.load(base_path+'/'+str(times)+'/begin.npy')
    fin = np.load(base_path+'/'+str(times)+'/fin.npy')
    

    synthetic_data = np.append(synthetic_data,begin,axis=1)
    for r in range(num):
        r_order = random.randint(1,times)
        r_index = random.randint(0,99)
        one_acc = np.load(base_path+'/'+str(times)+'/'+str(r_order)+'_'+str(r_index)+'.npy')
        synthetic_data = np.append(synthetic_data,one_acc,axis=1)    
    
    synthetic_data = np.append(synthetic_data,fin,axis=1)
    
    return synthetic_data

In [ ]:
def concat_data(subject,times):

    label = {}
    label['p'] = random.randint(1,20)
    label['s'] = random.randint(1,20)     
    label['a'] = random.randint(1,20)
    
    total_data = np.zeros((3,0))
    act_list = ['pushup','squat','abs']
    num_list = [label['p'],label['s'],label['a']]

    while any(num_list):

        work = random.randint(0,2)
        num_can = random.randint(1,20)
        act = act_list[work]
        num = min(num_list[work],num_can)
        data = synthesize_data(subject,act,num,times)
        num_list[work] = num_list[work] - num
        

        wide_r = random.uniform(0.8,1.2)
        height_r = random.uniform(0.8,1.2)
        length = data.shape[1]
        resample_length = np.arange(0,length,wide_r).shape[0]
        resample_data = np.zeros((3,resample_length))
        for d in range(3):
            resample_data[d] = np.interp(np.arange(0,length,wide_r),np.arange(0,length),data[d])
        scaled_data = resample_data*height_r
        shifted_data = np.zeros((3,resample_length))
        for s in range(3):
            shift_r = random.uniform(-2,2)
            shifted_data[s] = scaled_data[s] + shift_r
        

        total_data = np.append(total_data,shifted_data,axis=1)
    return total_data,label

### Parameter

In [ ]:
subject = 'A'
num_dataset = 5000

### Composite data creation

In [ ]:
X = np.zeros((0,3,35000))
labels = {'p':np.zeros(0),'s':np.zeros(0),'a':np.zeros(0)}

time1 = time.perf_counter()
for i in tqdm(range(num_dataset)):
    #合成データの作成
    data,label = concat_data(subject)
    if data.shape[1] <= 35000:
        #加速度データセットXに合成データを追加
        fix_data = np.zeros((1,3,35000))
        fix_data[0,:,:data.shape[1]] = data
        X = np.append(X,fix_data,axis=0)
        #ラベルの配列 y にラベルを追加
        for key in label:
            labels[key] = np.append(labels[key],label[key])
    
time2 = time.perf_counter()
print('time for making '+str(num_dataset)+' dataset : ',str(int((time2-time1)/60))+'min',str((time2-time1)%60)+'sec')

### Action for counting

In [ ]:
count_action = 'squat'
count_action_label = 's'

y = labels[count_action_label]

print(X.shape)
print(y.shape)

In [ ]:
plt.xticks(np.arange(21))
plt.hist(y,stacked=False,range=(0,21),bins=21,ec='k')

In [ ]:
ex = random.randint(0,999)
plt.figure(figsize=(30,5))
plt.plot(X[ex].T)
print(y[ex])

### Split the dataset into Train and Validation

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.8,random_state=1)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

### Test Data

In [ ]:
X_test = np.zeros((0,3,35000))
y_test = np.zeros(0)

test_path = '../data/raw_data/'+subject+'/'
files = os.listdir(test_path)
for file in  files:
    test_npz = np.load(test_path+file)
    test_data = test_npz['d']
    test_label = test_npz[count_action_label]
    
    fix_test_data = np.zeros((1,3,35000))
    fix_test_data[0,:,:test_data.shape[1]] = test_data
    
    X_test = np.append(X_test,fix_test_data,axis=0)
    y_test = np.append(y_test,test_label)
    
print(X_test.shape)
print(y_test.shape)

### Preprocessing

In [ ]:
def scaling(data):
    num_data = data.shape[0]
    for i in range(num_data):
        for j in range(3):
            data[i][j] = preprocessing.scale(data[i][j])
    return data

In [ ]:
X_train = scaling(X_train)
X_val = scaling(X_val)
X_test = scaling(X_test)

y_train = y_train/20
y_val = y_val/20
y_test = y_test/20

In [ ]:
X_train = X_train.transpose(0,2,1)
X_val = X_val.transpose(0,2,1)
X_test = X_test.transpose(0,2,1)

### Preparing pre-trained model

In [ ]:
hidden_unit = 8
kernel_size = 4

_input = Input(shape=(X_train.shape[1],X_train.shape[2]))
mask = Masking(mask_value=0.0)(_input)

conv_l = Conv1D(hidden_unit,8,padding='same',activation='relu')(mask)
conv_l = MaxPooling1D()(conv_l)
conv_l = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(conv_l)
conv_l = MaxPooling1D()(conv_l)
conv_l = Conv1D(hidden_unit,2,padding='same',activation='relu')(conv_l)
conv_l = MaxPooling1D()(conv_l)
conv_l = Dropout(0.5)(conv_l)

attention = Conv1D(hidden_unit,8,padding='same',activation='relu')(conv_l)
attention = Dropout(0.5)(attention)
attention = Conv1D(hidden_unit,8,padding='same',activation='relu')(attention)
attention = Activation('sigmoid')(attention)

sent_representation = Multiply()([conv_l, attention])
sent_representation = Dropout(0.5)(sent_representation)

x = Lambda(lambda xin: K.sum(xin,axis=2))(sent_representation)
x = Reshape((-1,1))(x)

x = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(x)
x = MaxPooling1D()(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(hidden_unit,activation='relu')(x)
_output = Dense(1,activation='linear')(x)

model = Model(inputs=_input, outputs=_output)
model.compile(loss='mean_absolute_error', optimizer='adam')

In [ ]:
model.load_weights('../data/pre-trained_model/wo_'+target+'_WeakCounter/100'+count_action+'.hdf5')

### Validation before Fine-Tuning (=WeakCounter)

In [ ]:
#テストデータ

pre_predict = model.predict(X_test)
pre_predict = pre_predict[:,0]

pre_MAE = mean_absolute_error(np.round(pre_predict*20),np.round(y_test*20))
print('MAE:',pre_MAE)
pre_MRE = np.mean(abs(np.round(pre_predict*20)-np.round(y_test*20))/np.round(y_test*20))
print('MRE:',pre_MRE)

In [ ]:

t = np.linspace(0,20,21)
s = t

plt.scatter(np.round(y_test*20),np.round(pre_predict*20))
plt.plot(s,t)
plt.xticks(range(0,21,2))
plt.yticks(range(0,21,2))
plt.grid()
plt.xlabel('Ground Truth')
plt.ylabel('Predict')

### Fine-Tuning

In [ ]:
Adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='mean_absolute_error',optimizer=Adam)

In [ ]:

nb_epoch = 50
plot_losses = keras_utils.PlotLosses()

start_train = time.perf_counter()

model.fit(X_train,y_train,epochs=nb_epoch,validation_data=(X_val,y_val),callbacks=[plot_losses],verbose=1)

end_train = time.perf_counter()

In [ ]:
print("Time for training:",str(int((end_train-start_train)/60))+'min',str((end_train-start_train)%60)+'sec')

### Prediction & Evaluation

In [ ]:
predict = model.predict(X_test)
predict = predict[:,0]
predict.shape

MAE = mean_absolute_error(np.round(predict*20),np.round(y_test*20))
print('MAE:',MAE)
accuracy = np.mean(1-abs(np.round(predict*20)-np.round(y_test*20))/np.round(y_test*20))
MRE = 1-accuracy
print('MRE:',MRE)

In [ ]:

t = np.linspace(0,20,21)
s = t

plt.scatter(np.round(y_test*20),np.round(predict*20))
plt.plot(s,t)
plt.xticks(range(0,21,2))
plt.yticks(range(0,21,2))
plt.grid()
plt.xlabel('Ground Truth')
plt.ylabel('Predict')